In [10]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

from otgnn.models import GCN
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model

from mol_opt2.mol_opt.data_mol_opt import MolOptDataset
from mol_opt2.mol_opt.data_mol_opt import get_loader
from mol_opt2.mol_opt.arguments import get_args
from mol_opt2.mol_opt.train_mol_opt import main, get_latest_model
from mol_opt2.mol_opt.ot_utils import compute_barycenter

from rdkit.Chem import MolFromSmiles

import torch
import numpy as np
import time

/home/octav/gitrepos/tum-thesis
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
sys.argv = ["", "-cuda", "-pred_hidden", "150", "-model_type", "ffn"]
args = get_args()
args.n_epochs = 5 
args.init_model = "slot2"
args.output_dir = "mol_opt/output_test"
args.tb_logs_dir = "mol_opt/logs_test"
args.init_decoder_model = "{}_decode".format(args.init_model)
args

Namespace(N_transformer=6, agg_func='sum', annealing_rate=0.05, batch_norm=False, connectivity=True, connectivity_hard=False, connectivity_lambda=0.015, cuda=True, device='cuda:0', dim_tangent_space=40, dropout_ffn=0.0, dropout_gcn=0.0, dropout_transformer=0.1, euler_characteristic_penalty=True, euler_lambda=0.2, ffn_activation='LeakyReLU', init_decoder_model='slot2_decode', init_model='slot2', linear_out=False, model_type='ffn', n_epochs=5, n_ffn_hidden=100, n_ffn_transformer=100, n_heads_transformer=10, n_hidden=50, n_labels=1, n_layers=5, ot_solver='emd', output_dir='mol_opt/output_test', pc_hidden=50, pred_hidden=150, sinkhorn_entropy=0.1, sinkhorn_max_it=10000, task='qed', tb_logs_dir='mol_opt/logs_test', valency=True, valency_hard=False, valency_lambda=0.05)

In [12]:
args.model_type

'ffn'

In [13]:
train_data_loader = get_loader("iclr19-graph2graph/data/qed", "train_pairs", 36, False, ratio = 0.0001)
val_data_loader = get_loader("iclr19-graph2graph/data/qed", "valid", 36, False, ratio = 0.01)

In [14]:
main(args, train_data_loader = train_data_loader, val_data_loader = val_data_loader)

Epoch: 6
idx_batch=0
 train_fgw:2.1992200
 train_conn_penalty:3.3052254
 train_val_penalty:1.4089796
 train_euler_penalty:5.3919589
 train_total:12.3053843
 train_similarity:0.0000000
 train_QED:0.4035316
 train_penlog:5.1543733
Epoch 6, train
 train_fgw:2.1992200
 train_conn_penalty:3.3052254
 train_val_penalty:1.4089796
 train_euler_penalty:5.3919589
 train_total:12.3053843
 train_similarity:0.0000000
 train_QED:0.4035316
 train_penlog:5.1543733
idx_batch=0
 val_fgw:1.4738704
 val_conn_penalty:-0.4574693
 val_val_penalty:2.0936681
 val_euler_penalty:1.3301734
 val_total:4.4402428
 val_similarity:0.0000000
 val_QED:0.3736734
 val_penlog:6.5553309
 val_molecular_disconnected_validity:100.0000000
 val_molecular_validity:0.0000000
 val_symbol_accuracy:78.9473684
 val_connected_components:25.3333333
 val_invalid_valency_nodes:0.0000000
Epoch 6, val
 val_fgw:1.4738704
 val_conn_penalty:-0.4574693
 val_val_penalty:2.0936681
 val_euler_penalty:1.3301734
 val_total:4.4402428
 val_similarity:0

(MolOpt(
   (GCN): GCN(
     (W_message_i): Linear(in_features=100, out_features=50, bias=False)
     (W_message_h): Linear(in_features=50, out_features=50, bias=False)
     (W_message_o): Linear(in_features=143, out_features=50, bias=True)
     (W_mol_h): Linear(in_features=50, out_features=100, bias=True)
     (W_mol_o): Linear(in_features=100, out_features=1, bias=True)
     (dropout_gcn): Dropout(p=0.0, inplace=False)
     (dropout_ffn): Dropout(p=0.0, inplace=False)
   )
   (opt0): Linear(in_features=50, out_features=50, bias=True)
   (opt1): Linear(in_features=50, out_features=50, bias=True)
 ),
 MolOptDecoder(
   (fc1_SYMBOLS): Linear(in_features=50, out_features=150, bias=True)
   (fc2_SYMBOLS): Linear(in_features=150, out_features=64, bias=True)
   (fc1_CHARGES): Linear(in_features=50, out_features=150, bias=True)
   (fc2_CHARGES): Linear(in_features=150, out_features=5, bias=True)
   (fc1_BONDS): Linear(in_features=100, out_features=150, bias=True)
   (fc2_BONDS): Linear(in_f